# Mistral 7b run after finetuning
In this notebook we run a previously finetuned version of Mistral 7b
* We created the dataset here: https://www.kaggle.com/code/alessandrodrago1/train-data-generation-for-llm-prompt-recovery
* We finetuned Mistral 7b here: https://www.kaggle.com/code/alessandrodrago1/mistral-7b-train

In [1]:
%%capture all
# https://www.kaggle.com/code/hotchpotch/llm-detect-pip 
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -Uq /kaggle/input/sentence-transformers-2-4-0/sentence_transformers-2.4.0-py3-none-any.whl
!pip install peft

In [2]:
import sys
import torch
import random
import numpy as np
import pandas as pd
import gc

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
sys.path.append("/kaggle/input/peft-main/src")

from peft import PeftModel

#https://github.com/Lightning-AI/lit-gpt/issues/327
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

In [3]:
#If Mistral doesn't respond - we use this well-scoring prompt instead...
#thanks to: https://www.kaggle.com/code/rdxsun/lb-0-61
base_line = 'Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.' 

#what's the longest prompt Mistral should respond with?
max_new_tokens = 35

# Load the model

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [5]:
HF_DATASETS_OFFLINE=1 
TRANSFORMERS_OFFLINE=1

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto
#base_model = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = "/kaggle/input/mistral-7b-it-v02"

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False 
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

# Load the finetuned model

In [8]:
model = PeftModel.from_pretrained(model, "/kaggle/input/mistral-7b-train/mistral_prompt_recovery_finetuned/")

In [9]:
def remove_numbered_list(text):
    final_text_paragraphs = [] 
    for line in text.split('\n'):
        # Split each line at the first occurrence of '. '
        parts = line.split('. ', 1)
        # If the line looks like a numbered list item, remove the numbering
        if len(parts) > 1 and parts[0].isdigit():
            final_text_paragraphs.append(parts[1])
        else:
            # If it doesn't look like a numbered list item, include the line as is
            final_text_paragraphs.append(line)

    return '  '.join(final_text_paragraphs)

#trims LLM output to just the response
def trim_to_response(text):
    terminate_string = "[/INST]"
    text = text.replace('</s>', '')
    #just in case it puts things in quotes
    text = text.replace('"', '')
    text = text.replace("'", '')

    last_pos = text.rfind(terminate_string)
    return text[last_pos + len(terminate_string):] if last_pos != -1 else text

#looks for response_start / returns only text that occurs after
def extract_text_after_response_start(full_text):
    parts = full_text.rsplit(response_start, 1)  # Split from the right, ensuring only the last occurrence is considered
    if len(parts) > 1:
        return parts[1].strip()  # Return text after the last occurrence of response_start
    else:
        return full_text  # Return the original text if response_start is not found
    
def trim_to_last_period(text):
    last_period_index = text.rfind('.')
    if last_period_index == -1:
        return text
    else:
        return text[:last_period_index + 1]

# Prompt setup
We prompt the model in the same way done during training

In [10]:
## original text prefix
orig_prefix = "Original Text:"

#mistral "response"
llm_response_for_rewrite = "Provide the modified text and I'll tell you the request that caused the change."

#modified text prefix
rewrite_prefix = "Re-written Text:"

#provided as start of Mistral response (anything after this is used as the prompt)
response_start = "The request was: "

In [11]:
def get_prompt(orig_text, transformed_text):

    #construct the prompt sequence...
    messages = [
        #actual prompt
        {"role": "user", "content": f"{orig_prefix} {orig_text}"},
        {"role": "assistant", "content": llm_response_for_rewrite},
        {"role": "user", "content": f"{rewrite_prefix} {transformed_text}"},
        {"role": "assistant", "content": response_start},
    ]
        
    #give it to Mistral
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = model_inputs.to("cuda") 
    generated_ids = model.generate(model_inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)

    #decode and trim to actual response
    decoded = tokenizer.batch_decode(generated_ids)
    just_response = trim_to_response(decoded[0])        
    final_text = extract_text_after_response_start(just_response)

    #default to baseline
    if len(final_text) == 0:
        final_text = base_line
    
    #mistral has been replying with numbered lists - clean them up....
    final_text = remove_numbered_list(final_text)
    final_text = trim_to_last_period(final_text)
    
    return final_text

# Evaluate 

In [12]:
test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")

for index, row in test_df.iterrows():
    result = get_prompt(row['original_text'], row['rewritten_text'])
    print(result)
    test_df.at[index, 'rewrite_prompt'] = result
    
test_df = test_df[['id', 'rewrite_prompt']]
test_df

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2024-08-28 10:03:14.417266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 10:03:14.417369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 10:03:14.542022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Make the text into a shanty song.


,id,rewrite_prompt
0,-1,Make the text into a shanty song.


In [13]:
test = pd.read_csv('/kaggle/input/llm-prompt-recovery-ground-truth-1/test (1).csv')
test = test.head(195)

In [14]:
for index, row in test.iterrows():
    result = get_prompt(row['original_text'], row['rewritten_text'])
    print(result)
    test.at[index, 'pred'] = result

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Make the text into a gourmet food festival brochure.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


 Describe this as an expertise to be gained.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

Make the text into an employee satisfaction survey.    **Answer:**    The text has been modified to create an employee satisfaction survey.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Make the text into a Shakespearean sonnet.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Make the text into a vintage carnival barkers pitch.
Make the text into a treasure map description.


In [15]:
test

,id,original_text,rewrite_prompt,rewritten_text,pred
0,40837062,South Wales Police officers had been pursuing ...,Make the text into a gourmet food festival bro...,## Gourmet Food Festival Brochure\n\n**Indulge...,Make the text into a gourmet food festival bro...
1,36917156,Police were told an Audi's number plates were ...,Describe this as an expertise to be gained.,The expertise gained in this scenario is the a...,Describe this as an expertise to be gained.
2,38155860,"Mr Burke, 19, from Ripon in North Yorkshire, w...",Change the text into an art critique,**Art Critique**\n\nThe text provided does not...,Im unable to complete the request as the text...
3,35189843,Twenty-four-year-old Aldis Minakovskis is alle...,Make the text into a detective's case file note,## Detective's Case File Note - Aldis Minakovs...,Make the text into a detectives case file note.
4,36670719,The 29-year-old has signed a four-year contrac...,Convert this into a financial forecast report.,**Financial Forecast Report**\n\n**Company:** ...,Make the text into a financial forecast report...
...,...,...,...,...,...
190,32399004,"Andrew Picton, 66, of Eastcroft Close, Blagdon...",Craft this into a business continuity plan.,## Business Continuity Plan for Company XYZ\n\...,Make this a business continuity plan. The t...
191,40964641,17 August 2017 Last updated at 15:29 BST\nA so...,Transform this into an employee satisfaction s...,**Employee Satisfaction Survey**\n\n**Please a...,Make the text into an employee satisfaction su...
192,34703934,The group will play at the Etihad Stadium in t...,Transform the message into a Shakespearean sonnet,"O sweet band, thy music doth evoke,\nA tale of...",Make the text into a Shakespearean sonnet.
193,38423051,Fans protesting at City's owners Sisu held the...,Turn the content into an old-timey carnival an...,"Gather 'round, folks, and hear what I have to ...",Make the text into a vintage carnival barkers ...


In [16]:
from tqdm import tqdm
tqdm.pandas()

import warnings 
warnings.filterwarnings('ignore')

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
def CVScore(test):
    
    scs = lambda row: abs((cosine_similarity(row["actual_embeddings"], row["pred_embeddings"])) ** 3)
    
    model = SentenceTransformer('/kaggle/input/sentence-t5-base-hf/sentence-t5-base')
    
    print(test["rewrite_prompt"])

    test["actual_embeddings"] = test["rewrite_prompt"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))
    print(test["actual_embeddings"])
    test["pred_embeddings"] = test["pred"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))
    
    test["score"] = test.apply(scs, axis=1)
    
    return np.mean(test['score'])[0][0]
    
print(f"CV Score: {CVScore(test)}")

0      Make the text into a gourmet food festival bro...
1            Describe this as an expertise to be gained.
2                   Change the text into an art critique
3        Make the text into a detective's case file note
4         Convert this into a financial forecast report.
                             ...                        
190          Craft this into a business continuity plan.
191    Transform this into an employee satisfaction s...
192    Transform the message into a Shakespearean sonnet
193    Turn the content into an old-timey carnival an...
194             Translate this into an adventurer's map.
Name: rewrite_prompt, Length: 195, dtype: object


100%|██████████| 195/195 [00:02<00:00, 73.93it/s]


0      [[-0.020476142, -0.017723111, -0.0105812475, 0...
1      [[-0.018977212, -0.0053801383, 0.016442953, 0....
2      [[-0.039775047, -0.002232923, 0.009845024, 0.0...
3      [[-0.021092517, -0.034007005, 0.0015480592, 0....
4      [[-0.0048643555, -0.013603131, -0.031679187, 0...
                             ...                        
190    [[0.020877542, -0.0077270623, -0.0050881705, 0...
191    [[-0.01017829, -0.01296109, 0.0070637036, 0.03...
192    [[-0.035690114, -0.003417209, 0.023152454, 0.0...
193    [[-0.022775475, -0.03535786, -0.03330925, 0.04...
194    [[-0.019695086, 0.011401464, 0.03006828, 0.051...
Name: actual_embeddings, Length: 195, dtype: object


100%|██████████| 195/195 [00:02<00:00, 76.74it/s]


CV Score: 0.7554425597190857


# Submit

In [18]:
test_df.to_csv('submission.csv', index=False)